<a href="https://colab.research.google.com/github/IamArmanNikkhah/Air-Traffic-Management/blob/main/ATM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class FlightPlan:
  def __init__(self,aa:int, fn:int, fd:int, rf:int, sp:bool):
    if aa >= 0 and fd >= 0 and rf > 0 and type(sp)==bool and type(fn) == int:
      self.aaet = aa
      self.flight_duration = fd
      self.rFuel = rf
      self.specialP =sp
      self.flight_number = fn
    else:
      print("FlightPlan: Check your input!")

In [ ]:
onBound_TIME = 0 # hypothesis-always constant
class ATM:
  def __init__(self, SCH_algh:str):
    alghorithms_ = ['FCFS','non_LFDF','non_LRFF','LFDF','LRFF']
    self.scheduling_algh = next(algh for algh in alghorithms_ if algh == SCH_algh)
    self.timer = 0
    self.all_list = []
    self.waiting_list_normal = []
    self.waiting_list_vip = []
    self.running = None
    self.history = []
  
  def setFlightPlan(self,FP_list):
    self.all_list = FP_list
  def addFlightPlan(self,FP_item):
    self.all_list.append(FP_item)
  
  def addtimer(self,t = 1):
    self.timer += t
    for plan in self.all_list.copy():
      if plan.aaet <= self.timer and not plan.specialP:
        self.waiting_list_normal.append(plan)
        self.all_list.remove(plan)

      elif plan.aaet <= self.timer and plan.specialP:
        self.waiting_list_vip.append(plan)
        self.all_list.remove(plan)


  def run(self, fp, cond_func):
    self.running = fp
    self.history.append("running process : " + str(self.running.flight_number) + ' at time :' + str(self.timer))

    
    if fp in self.waiting_list_normal: 
      wlist = self.waiting_list_normal
      if self.waiting_list_vip: self.history.append("VIP list is not empty!!")
    elif fp in self.waiting_list_vip: wlist = self.waiting_list_vip
    else: print("ATM : error running process -> doesnt exist")
    cond = cond_func(fp, wlist)
    for i in range (0,onBound_TIME + 1) : ###
      if not cond:
        break
      self.addtimer()
      cond = cond_func(fp, wlist)
    self.running = None
    if i == onBound_TIME and cond:
      wlist.remove(fp)
      self.history.append(" process : " + str(fp.flight_number) + ' finished at time :' + str(self.timer))
      return True
    else:
      self.history.append("process : " + str(fp.flight_number) + "Blocked!")
      return False


  def noCOND(fp, Wlist):
    return True
  
  def lfCOND(fp, Wlist):
    for item in Wlist:
      if item.flight_duration > fp.flight_duration:
        return False
    return True

  def lrCOND(fp, Wlist):
    for item in Wlist:
      if item.rFuel < fp.rFuel:
        return False
    return True
  
  def FCFS(self, cond_func = noCOND):
    
    while self.waiting_list_normal or self.waiting_list_vip:
      self.waiting_list_normal.sort(reverse = False, key = lambda x: x.flight_number)
      self.waiting_list_vip.sort(reverse = False, key = lambda x: x.flight_number)
      if self.waiting_list_vip:
        self.run(next(iter(self.waiting_list_vip)), cond_func)
      else:
        self.run(next(iter(self.waiting_list_normal)), cond_func)
    

  def non_LFDF(self, cond_func = noCOND):
    while self.waiting_list_normal or self.waiting_list_vip:
      self.waiting_list_vip.sort(reverse = True, key = lambda x: x.flight_duration)
      self.waiting_list_normal.sort(reverse = True, key = lambda x: x.flight_duration)
      if self.waiting_list_vip:
        self.run(next(iter(self.waiting_list_vip)), cond_func)
      else:
        self.run(next(iter(self.waiting_list_normal)), cond_func)  

  def non_LRFF(self, cond_func = noCOND):
    while self.waiting_list_normal or self.waiting_list_vip:
      self.waiting_list_normal.sort(reverse = False, key = lambda x: x.rFuel)
      self.waiting_list_vip.sort(reverse = False, key = lambda x: x.rFuel)
      if self.waiting_list_vip:
        self.run(next(iter(self.waiting_list_vip)), cond_func)
      else:
        self.run(next(iter(self.waiting_list_normal)), cond_func)

  def LFDF(self, cond_func = lfCOND):
    self.non_LFDF(cond_func)

  def LRFF(self, cond_func = lrCOND):
    self.non_LRFF(cond_func)


  def start(self):
    while self.all_list or self.waiting_list_normal or self.waiting_list_vip:
      if not self.waiting_list_normal and not self.waiting_list_vip:
        self.addtimer()
      self.FCFS() # for running diffrent alghorithm use : self.non_LFDF() OR  self.non_LRFF() OR self.LFDF()  OR  self.LRFF()
    print(self.history)



In [ ]:
F1 = FlightPlan(2, 553, 20, 10, False)
F2 = FlightPlan(5, 445, 15, 12, False)
F3 = FlightPlan(5, 212, 50, 10, True)
F4 = FlightPlan(5, 788, 11, 15, False)
F5 = FlightPlan(7, 600, 4, 1, True)
F6 = FlightPlan(8, 620, 2, 14, False)
f_list = {F1, F2, F3, F4, F5, F6}

In [ ]:
Sys = ATM('FCFS')
Sys.setFlightPlan(f_list)
Sys.start()

['running process : 553 at time :2']
['running process : 553 at time :2', ' process : 553 finished at time :3', 'running process : 212 at time :5']
['running process : 553 at time :2', ' process : 553 finished at time :3', 'running process : 212 at time :5', ' process : 212 finished at time :6', 'running process : 445 at time :6']
['running process : 553 at time :2', ' process : 553 finished at time :3', 'running process : 212 at time :5', ' process : 212 finished at time :6', 'running process : 445 at time :6', ' process : 445 finished at time :7', 'running process : 600 at time :7']
['running process : 553 at time :2', ' process : 553 finished at time :3', 'running process : 212 at time :5', ' process : 212 finished at time :6', 'running process : 445 at time :6', ' process : 445 finished at time :7', 'running process : 600 at time :7', ' process : 600 finished at time :8', 'running process : 620 at time :8']
['running process : 553 at time :2', ' process : 553 finished at time :3', 